<a id='1.3'></a>
## Import Packages

In [7]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import datetime

from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from stable_baselines3.common.logger import configure
from finrl.meta.data_processor import DataProcessor
from sklearn.preprocessing import StandardScaler #平均值是0, 標準差是1
from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from pprint import pprint

import sys
sys.path.append("../FinRL")

import itertools

c:\Users\yaowe\anaconda3\envs\myenv\lib\site-packages\pyfolio\pos.py:26: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  warnings.warn(


<a id='1.4'></a>
## Create Folders

In [8]:
from finrl import config
from finrl import config_tickers
import os
from finrl.main import check_and_make_directories
from finrl.config import (
    DATA_SAVE_DIR,
    TRAINED_MODEL_DIR,
    TENSORBOARD_LOG_DIR,
    RESULTS_DIR,
    INDICATORS,
    TRAIN_START_DATE,
    TRAIN_END_DATE,
    TEST_START_DATE,
    TEST_END_DATE,
    TRADE_START_DATE,
    TRADE_END_DATE,
    CHIPS,
)
check_and_make_directories([DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR])



In [9]:
#1. Q1 季報：5 月 15 日前
#2. Q2 季報：8 月 14 日前
#3. Q3 季報：11 月 14 日前
#4. 年度財務報告：次年 3 月 31 日前
#7
TRAIN_START_DATE = '2013-01-01'
TRAIN_END_DATE = '2023-08-16'
TRADE_START_DATE = '2023-08-16'
TRADE_END_DATE = '2023-11-16'

#8
TRAIN_START_DATE = '2013-01-01'
TRAIN_END_DATE = '2023-11-16'
TRADE_START_DATE = '2023-11-16'
TRADE_END_DATE = '2024-04-01'

#19
TRAIN_START_DATE = '2013-01-01'
TRAIN_END_DATE = '2024-05-17'
TRADE_START_DATE = '2024-05-17'
TRADE_END_DATE = '2024-08-17'

In [11]:
df = pd.read_csv('tcn_daily_trade_info/tcn_daily_trade_info.csv')
df                           

,date,tic,close,coding1,coding2,coding3,coding4,coding5,coding6,coding7,...,coding11,coding12,coding13,coding14,coding15,coding16,coding17,coding18,coding19,coding20
0,2013-02-06,2330,105.0,0.366821,0.841420,-0.154679,0.058533,-0.230075,0.021644,0.796686,...,-0.633320,0.455320,0.820700,-0.660500,0.447149,0.542694,1.465947,0.632877,0.247811,0.123082
1,2013-02-19,2330,107.0,0.412771,0.823482,-0.169980,0.090850,-0.291289,-0.052444,0.777444,...,-0.553663,0.546829,0.820707,-0.526450,0.389043,0.538590,1.419131,0.638307,0.263613,0.166649
2,2013-02-20,2330,109.0,0.499778,0.811340,-0.171635,0.172657,-0.457755,-0.090967,0.783952,...,-0.534661,0.582521,0.813045,-0.412073,0.373875,0.505340,1.334784,0.629095,0.253826,0.207985
3,2013-02-21,2330,107.0,0.510544,0.865459,-0.161747,0.284643,-0.480295,-0.089888,0.789663,...,-0.613607,0.598486,0.822012,-0.534205,0.482969,0.451621,1.258745,0.648102,0.248005,0.271877
4,2013-02-22,2330,105.5,0.459863,0.921646,-0.176326,0.375388,-0.352707,-0.110306,0.804166,...,-0.673164,0.638396,0.815272,-0.701006,0.622197,0.384897,1.243376,0.675095,0.247471,0.349467
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156076,2024-05-29,2383,427.0,0.531800,-0.217562,-0.207273,0.270437,-0.533960,0.196958,0.536517,...,-0.381020,0.547690,1.508795,-0.720522,0.851125,0.699900,1.961354,0.439108,1.491817,0.788919
156077,2024-05-30,2383,415.5,0.461620,-0.205481,-0.248325,0.270308,-0.446093,0.161119,0.503514,...,-0.511669,0.544506,1.519936,-0.775979,0.812284,0.678520,1.975939,0.427805,1.492989,0.788479
156078,2024-05-31,2383,408.5,0.387324,-0.182236,-0.263486,0.251943,-0.258898,0.109919,0.496318,...,-0.600187,0.562166,1.538140,-0.730638,0.779476,0.656927,1.995894,0.428045,1.469981,0.806247
156079,2024-06-04,2383,430.5,0.349762,-0.126509,-0.235017,0.212807,-0.131768,0.046090,0.488470,...,-0.566339,0.562096,1.541757,-0.596351,0.741829,0.664549,2.036993,0.414560,1.465102,0.764983


In [12]:
train = data_split(df, TRAIN_START_DATE,TRAIN_END_DATE)
trade = data_split(df, TRADE_START_DATE,TRADE_END_DATE)
print(len(train))
print(len(trade))

155267
814


In [14]:
df = pd.read_csv('tcn_daily_trade_info/tcn_daily_trade_info.csv')

#過濾只要交易的股票
csv_file_path = 'Low-risk stock list.csv'
df_mask = pd.read_csv(csv_file_path)
first_row = df_mask.iloc[10] #讀取哪一季的股票

stocks_with_1 = first_row[first_row == 1].index.tolist()

# 移除每個代碼中的 '.TW'
stocks_without_suffix = [stock.split('.')[0] for stock in stocks_with_1]

print(stocks_without_suffix)

filter_tics=[1229, 2324, 2356, 2376, 2382, 2395, 2610, 2618, 3037, 3443, 3533, 8046, 9941, 9945]
df = df[df['tic'].isin(filter_tics)]
df = data_split(df, TRAIN_START_DATE,TRADE_END_DATE)
#過濾收盤價是0
df = df[df['close'] != 0]
    

df_cash_share = pd.read_csv('twse_divide_ratio_TCN.csv')
    
df_cash_share = df_cash_share[df_cash_share['tic'].isin(filter_tics)]
# 創建一個字典來存儲每個元素在filter_tics中的索引
index_dict = {tic: index for index, tic in enumerate(filter_tics)}

# 使用sorted函數根據filter_tics的索引排序列表
sorted_list = sorted(filter_tics, key=lambda x: index_dict.get(x, float('inf')))
df_cash_share['index'] = df_cash_share['tic'].apply(lambda x: sorted_list.index(x)+1  if x in sorted_list else None)

    
df = pd.merge(df, df_cash_share, how = 'left', on=["date","tic"])
df = df.fillna(value=0)


['1229', '2324', '2356', '2376', '2382', '2395', '2610', '2618', '3037', '3443', '3533', '8046', '9941', '9945']


In [15]:
df.sort_values(['date','tic'],ignore_index=True).head()

df = df.fillna(method='ffill')
df

C:\Users\yaowe\AppData\Local\Temp\ipykernel_18168\414903098.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill')


,date,tic,close,coding1,coding2,coding3,coding4,coding5,coding6,coding7,...,coding14,coding15,coding16,coding17,coding18,coding19,coding20,share,cash,index
0,2013-02-06,2324,20.75,0.531713,0.498987,-0.224538,0.055445,-0.193646,-0.137709,0.762302,...,-0.456326,0.827445,0.278539,0.976848,0.632865,0.112394,0.377188,0.0,0.0,0.0
1,2013-02-06,2382,68.00,0.685717,0.585296,0.162812,0.556193,-0.365952,0.071033,0.702315,...,-0.400978,0.569046,0.231672,1.426906,0.662290,0.649988,0.329348,0.0,0.0,0.0
2,2013-02-06,2395,130.50,0.436256,0.921688,-0.346031,0.156141,-0.209958,-0.070421,0.586787,...,-0.621555,0.329642,0.466234,1.075748,0.789490,0.210285,0.212214,0.0,0.0,0.0
3,2013-02-06,3037,28.50,0.134630,0.636879,0.102470,0.330109,-0.212586,0.012381,0.696222,...,-0.543619,0.203265,0.292264,1.238429,0.360958,0.543075,-0.029863,0.0,0.0,0.0
4,2013-02-06,3443,93.80,0.257116,0.519377,0.152827,0.541301,-0.180910,-0.068960,0.723473,...,-0.513987,0.233457,0.188001,1.411778,0.649515,0.616597,0.184437,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29612,2024-06-05,3443,1580.00,0.658868,0.144944,-0.336764,0.221048,-0.081059,0.061236,0.755844,...,-0.853171,0.812033,0.671617,1.933807,0.929106,1.569586,0.768042,0.0,0.0,0.0
29613,2024-06-05,3533,1595.00,0.437267,0.076646,-0.472099,0.134413,-0.246587,0.139379,0.952172,...,-0.951985,0.941479,0.660762,1.678192,1.046278,1.699484,0.738302,0.0,0.0,0.0
29614,2024-06-05,8046,191.00,0.764475,-0.227007,-0.334844,0.193825,-0.492236,0.000845,0.332980,...,-0.642271,0.736638,0.879485,1.728126,0.342086,1.628090,0.917597,0.0,0.0,0.0
29615,2024-06-05,9941,150.00,0.315398,0.051921,-0.219945,0.520131,-0.282850,0.118166,0.055047,...,-0.691445,0.533069,0.984331,1.672904,0.061797,1.708297,0.695229,0.0,0.0,0.0


In [16]:
#驗證clean Data
df1 = df.copy()
df1 = df1.sort_values(["date", "tic"], ignore_index=True)

df1.index = df1.date.factorize()[0]

merged_closes = df1.pivot_table(index="date", columns="tic", values="close")

columns_with_nan = merged_closes.columns[merged_closes.isna().any()].tolist()

rows_with_nan = merged_closes[merged_closes.isna().any(axis=1)]

rows_with_nan

tic,1229,2324,2356,2376,2382,2395,2610,2618,3037,3443,3533,8046,9941,9945
date,,,,,,,,,,,,,,
2013-02-06,NaN,20.75,NaN,NaN,68.0,130.5,NaN,NaN,28.50,93.8,NaN,35.25,72.4,NaN
2013-02-19,NaN,20.50,11.95,26.05,66.6,133.0,12.35,18.05,28.40,94.7,88.7,36.10,73.1,67.0
2013-03-07,NaN,20.45,NaN,NaN,64.5,131.5,NaN,NaN,28.35,94.1,NaN,34.00,78.7,67.7
2013-04-25,NaN,18.80,NaN,NaN,58.8,135.0,NaN,NaN,29.50,89.0,NaN,33.50,74.9,NaN
2013-07-26,NaN,20.00,NaN,NaN,71.0,149.5,NaN,NaN,28.65,96.7,NaN,39.70,75.5,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-15,63.0,31.80,52.10,271.50,233.0,351.0,22.45,30.65,190.50,1480.0,901.0,272.00,182.0,NaN
2023-09-19,61.7,30.95,48.50,253.50,219.0,345.0,22.20,30.25,188.50,1425.0,874.0,271.00,182.0,NaN
2023-09-20,61.8,30.55,48.50,263.50,218.5,342.5,22.15,29.95,183.50,1360.0,853.0,260.50,180.5,NaN


In [17]:
columns_to_check=[1229, 2324, 2356, 2376, 2382, 2395, 2610, 2618, 3037, 3443, 3533, 8046, 9941, 9945]


if 'date' not in df.index.names:
    df.set_index('date', inplace=True)
    print("Date set as index in df")


for col in columns_to_check:
    if col in rows_with_nan.columns:
        
        nan_dates = rows_with_nan[rows_with_nan[col].isna()].index.tolist()
        #print(f"NaN Dates for column {col}:", nan_dates)
        
        df = df.drop(nan_dates, errors='ignore')
        #print(f"Rows corresponding to NaN dates for column {col} have been removed.")

df.reset_index(inplace=True)
print("All operations complete.")


Date set as index in df
All operations complete.


In [18]:
df.drop_duplicates(subset=['date', 'tic'], keep='first', inplace=True)

In [19]:
# 檢查每個 'tic' 組内的 'date' 是否有重複
duplicates = df.duplicated(subset=['tic', 'date'], keep=False)
if duplicates.any():
    print("There are duplicates within groups:")
    print(df[duplicates])
else:
    print("No duplicates found within groups.")

No duplicates found within groups.


In [20]:
fe = FeatureEngineer(
                    use_technical_indicator=False,
                    tech_indicator_list = False,
                    use_vix=False,
                    use_turbulence=False, 
                    user_defined_feature = False)

processed = fe.preprocess_data(df)

c:\Users\yaowe\Desktop\Personal\code\PMvRLnGAN_web_v5\PMvRLnGAN\Trading Agent\finrl\meta\preprocessor\preprocessors.py:105: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method="ffill").fillna(method="bfill")


In [21]:
list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
combination = list(itertools.product(list_date,list_ticker))

processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])

processed_full = processed_full.fillna(0)

In [22]:
processed_full.sort_values(['tic','date'],ignore_index=True).tail(10)

,date,tic,close,coding1,coding2,coding3,coding4,coding5,coding6,coding7,...,coding14,coding15,coding16,coding17,coding18,coding19,coding20,share,cash,index
28900,2024-05-21,9945,44.35,0.785543,-0.168655,-0.505213,0.114634,-0.263251,0.045374,0.300308,...,-0.361325,1.026956,0.811807,1.613456,0.291234,1.043413,0.863646,0.0,0.0,0.0
28901,2024-05-22,9945,44.70,0.765467,-0.190472,-0.505594,0.017541,-0.326463,0.021707,0.299032,...,-0.325892,0.970264,0.857479,1.608787,0.326604,1.030012,0.931142,0.0,0.0,0.0
28902,2024-05-23,9945,43.25,0.708095,-0.121200,-0.479180,-0.088396,-0.327563,-0.023404,0.266030,...,-0.445474,0.984143,0.889690,1.602738,0.356901,1.017741,0.956101,0.0,0.0,0.0
28903,2024-05-24,9945,43.65,0.628768,-0.041157,-0.467550,-0.135705,-0.352675,-0.079341,0.212537,...,-0.639740,0.981992,0.883550,1.586081,0.360959,1.045282,0.914435,0.0,0.0,0.0
28904,2024-05-28,9945,45.40,0.519719,0.034980,-0.492656,-0.124088,-0.366181,-0.102550,0.161825,...,-0.739788,0.993933,0.877468,1.572127,0.335171,1.056028,0.847819,0.0,0.0,0.0
28905,2024-05-29,9945,44.55,0.428063,0.065725,-0.553880,-0.086380,-0.303415,-0.126859,0.117315,...,-0.754717,0.976269,0.849634,1.549565,0.322826,1.070448,0.802574,0.0,0.0,0.0
28906,2024-05-30,9945,43.85,0.396305,0.005331,-0.610261,-0.049697,-0.221006,-0.144031,0.090426,...,-0.616777,0.966833,0.826887,1.516605,0.313185,1.080532,0.802548,0.0,0.0,0.0
28907,2024-05-31,9945,44.60,0.365080,-0.023964,-0.602050,-0.037479,-0.102250,-0.171273,0.097782,...,-0.445716,0.965806,0.867319,1.487000,0.345289,1.097403,0.799251,0.0,0.0,0.0
28908,2024-06-04,9945,44.90,0.399233,-0.020609,-0.524605,-0.073281,-0.037002,-0.187523,0.106254,...,-0.378301,0.940175,0.946769,1.478591,0.379334,1.112235,0.773493,0.0,0.0,0.0
28909,2024-06-05,9945,45.15,0.461046,-0.053319,-0.466302,-0.102498,-0.087080,-0.189561,0.128252,...,-0.468245,0.909648,1.034319,1.459677,0.431069,1.131442,0.712935,0.0,0.0,0.0


In [23]:
train = data_split(processed_full, TRAIN_START_DATE,TRAIN_END_DATE)
trade = data_split(processed_full, TRADE_START_DATE,TRADE_END_DATE)
print(len(train))
print(len(trade))

28756
154


In [24]:
unique_train_dates_count = train['date'].nunique()
unique_trade_dates_count = trade['date'].nunique()
print(unique_train_dates_count) 
print(unique_trade_dates_count)

2054
11


In [25]:
print(CHIPS)
print(INDICATORS)

[]
['coding1', 'coding2', 'coding3', 'coding4', 'coding5', 'coding6', 'coding7', 'coding8', 'coding9', 'coding10', 'coding11', 'coding12', 'coding13', 'coding14', 'coding15', 'coding16', 'coding17', 'coding18', 'coding19', 'coding20']


In [26]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension + len(CHIPS)*stock_dimension 
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 14, State Space: 309


In [27]:
train.tic.unique()

array([1229, 2324, 2356, 2376, 2382, 2395, 2610, 2618, 3037, 3443, 3533,
       8046, 9941, 9945], dtype=int64)

In [28]:
df_csp = pd.read_csv('csp_TCN.csv')
df_csp = df_csp[df_csp['tic'].isin(filter_tics)]
# 創建一個字典來存儲每個元素在filter_tics中的索引
index_dict = {tic: index for index, tic in enumerate(filter_tics)}

# 使用sorted函數根據filter_tics的索引排序列表
sorted_list = sorted(filter_tics, key=lambda x: index_dict.get(x, float('inf')))
df_csp['index'] = df_csp['tic'].apply(lambda x: sorted_list.index(x) +1 if x in sorted_list else None)

    
df_csp

,tic,stock_id,date,new_stock,return,index
7,9945,潤泰新,2018-10-08,600.0,4.0,14
15,9945,潤泰新,2023-09-25,900.0,1.0,14


In [29]:
buy_cost_list =  [0.001425] * stock_dimension
sell_cost_list = [0.004425] * stock_dimension
num_stock_shares = [0] * (stock_dimension)

In [30]:
env_kwargs = {
    "hmax": 1000,
    "initial_amount": 1000000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension, # new action才要+1
    "reward_scaling": 1e-4,
    "chips_list": CHIPS,
    "cap_reduction" : df_csp,
    "cash_share" : df_cash_share,
}


e_train_gym = StockTradingEnv(df = train, **env_kwargs)

In [31]:
obs = e_train_gym.reset()
len(obs)

309

print("Initial observation:", obs)
for _ in range(10):  # 模拟一系列动作
    action = e_train_gym.action_space.sample()
    obs, _, _, _ = e_train_gym.step(action)
    print("Observation after action:", obs)

In [32]:
e_train_gym

In [33]:
print(num_stock_shares)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [35]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


c:\Users\yaowe\anaconda3\envs\myenv\lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


In [36]:
agent = DRLAgent(env = env_train)


### Agent 3: PPO

In [38]:

df_account_value_ppo = pd.DataFrame()
df_balance_ppo = pd.DataFrame()
df_actions_ppo = pd.DataFrame()

agent = DRLAgent(env = env_train)
PPO_PARAMS = {
    "n_steps": 2048,
    "ent_coef": 0.01,
    "learning_rate": 0.00025,
    "batch_size": 250,
}
model_ppo = agent.get_model("ppo")

# set up logger
tmp_path = RESULTS_DIR + '/ppo'
new_logger_ppo = configure(tmp_path, ["stdout", "csv", "tensorboard"])
# Set new logger
model_ppo.set_logger(new_logger_ppo)
        
trained_ppo = agent.train_model(model=model_ppo, 
                         tb_log_name='ppo',
                         total_timesteps=5000)  

# 保存訓練好的模型
import os
import json

# 創建 models 目錄（如果不存在）
models_dir = "models"
if not os.path.exists(models_dir):
    os.makedirs(models_dir)

# 保存模型
model_save_path = os.path.join(models_dir, "trading_agent_model")
trained_ppo.save(model_save_path)
print(f"模型已保存到 {model_save_path}")

# 保存模型配置
model_config = {
    "policy": "MlpPolicy",
    "learning_rate": 0.00025,
    "batch_size": 250,
    "ent_coef": 0.01,
    "n_steps": 2048,
    "gamma": 0.99,
    "train_timesteps": 5000,
    # 只保存基本環境信息
    "env_info": {
        "env_class": "StockTradingEnv",
        "stock_dim": e_train_gym.stock_dim if hasattr(e_train_gym, "stock_dim") else None,
        "initial_amount": getattr(e_train_gym, "initial_amount", 1000000)
    }
}

config_save_path = os.path.join(models_dir, "trading_agent_config.json")
with open(config_save_path, "w") as f:
    json.dump(model_config, f, indent=4)
print(f"模型配置已保存到 {config_save_path}")

e_trade_gym = StockTradingEnv(df = trade, turbulence_threshold = None,risk_indicator_col='vix', **env_kwargs)


trained_moedl =trained_ppo
df_account_value, df_actions ,df_balance= DRLAgent.DRL_prediction(
    model=trained_moedl, 
    environment = e_trade_gym)
        
print("==============Get Backtest Results===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')
        
perf_stats_all = backtest_stats(account_value=df_account_value)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+RESULTS_DIR+"/perf_stats_all_"+now+'.csv')



df_account_value_ppo.to_csv("./"+RESULTS_DIR+"/account_ppo_10L5N_1_"+now+'.csv')
df_balance_ppo.to_csv("./"+RESULTS_DIR+"/balance_ppo_10L5N_1_"+now+'.csv')
df_actions_ppo.to_csv("./"+RESULTS_DIR+"/action_ppo_10L5N_1_"+now+'.csv')

# 保存交易決策示例
decisions_save_path = os.path.join(models_dir, "trading_decisions_examples.csv")
df_actions.head(30).to_csv(decisions_save_path, index=True)
print(f"交易決策示例已保存到 {decisions_save_path}")

# 保存性能指標
performance_metrics = {
    "annual_return": float(perf_stats_all.loc["Annual return"].values[0]),
    "cumulative_returns": float(perf_stats_all.loc["Cumulative returns"].values[0]),
    "annual_volatility": float(perf_stats_all.loc["Annual volatility"].values[0]),
    "sharpe_ratio": float(perf_stats_all.loc["Sharpe ratio"].values[0]),
    "calmar_ratio": float(perf_stats_all.loc["Calmar ratio"].values[0]),
    "stability": float(perf_stats_all.loc["Stability"].values[0]),
    "max_drawdown": float(perf_stats_all.loc["Max drawdown"].values[0]),
    "omega_ratio": float(perf_stats_all.loc["Omega ratio"].values[0]),
    "sortino_ratio": float(perf_stats_all.loc["Sortino ratio"].values[0]),
    "skew": float(perf_stats_all.loc["Skew"].values[0]) if not pd.isna(perf_stats_all.loc["Skew"].values[0]) else 0,
    "kurtosis": float(perf_stats_all.loc["Kurtosis"].values[0]) if not pd.isna(perf_stats_all.loc["Kurtosis"].values[0]) else 0,
    "tail_ratio": float(perf_stats_all.loc["Tail ratio"].values[0]),
    "daily_value_at_risk": float(perf_stats_all.loc["Daily value at risk"].values[0])
}

performance_save_path = os.path.join(models_dir, "trading_agent_performance.json")
with open(performance_save_path, "w") as f:
    json.dump(performance_metrics, f, indent=4)
print(f"性能指標已保存到 {performance_save_path}")


MODEL_KWARGS:  {'ppo': {'n_steps': 4096, 'ent_coef': 0.01, 'learning_rate': 0.00025, 'batch_size': 4096}}
model_kwargs:  {'n_steps': 4096, 'ent_coef': 0.01, 'learning_rate': 0.00025, 'batch_size': 4096}
Get Model kwargs:  {'n_steps': 4096, 'ent_coef': 0.01, 'learning_rate': 0.00025, 'batch_size': 4096}
Using cuda device
Logging to results/ppo


c:\Users\yaowe\anaconda3\envs\myenv\lib\site-packages\stable_baselines3\common\on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


--------------------------------
| time/              |         |
|    fps             | 269     |
|    iterations      | 1       |
|    time_elapsed    | 15      |
|    total_timesteps | 4096    |
| train/             |         |
|    reward          | 0.05944 |
--------------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 272          |
|    iterations           | 2            |
|    time_elapsed         | 30           |
|    total_timesteps      | 8192         |
| train/                  |              |
|    approx_kl            | 0.0034274973 |
|    clip_fraction        | 0.00569      |
|    clip_range           | 0.2          |
|    entropy_loss         | -19.9        |
|    explained_variance   | -0.0809      |
|    learning_rate        | 0.00025      |
|    loss                 | 1.5          |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.00723     |
|    reward    

c:\Users\yaowe\anaconda3\envs\myenv\lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


In [39]:
df_actions

,1229,2324,2356,2376,2382,2395,2610,2618,3037,3443,3533,8046,9941,9945
date,,,,,,,,,,,,,,
2024-05-17,48,0,0,0,0,110,177,0,0,0,63,0,0,0
2024-05-21,48,0,0,0,0,110,177,0,0,0,63,0,0,0
2024-05-22,48,0,0,0,0,110,177,0,0,0,63,0,0,0
2024-05-23,48,0,0,0,0,110,177,0,0,0,63,0,0,0
2024-05-24,48,0,0,0,0,110,177,0,0,0,63,0,0,0
2024-05-28,48,0,0,0,0,110,177,0,0,0,63,0,0,0
2024-05-29,19,0,0,0,0,109,168,0,0,0,29,0,0,0
2024-05-30,-7,0,0,0,1,0,1,0,0,0,0,1,0,0
2024-05-31,-2,0,0,0,0,0,0,0,0,0,0,0,0,3


In [40]:
trade

,date,tic,close,coding1,coding2,coding3,coding4,coding5,coding6,coding7,...,coding14,coding15,coding16,coding17,coding18,coding19,coding20,share,cash,index
0,2024-05-17,1229,68.00,0.675061,0.042463,-0.195831,0.628255,-0.299507,-0.073044,0.268707,...,-0.564263,0.933504,0.628834,1.685006,-0.032120,1.466374,0.722460,0.0,0.0,0.0
0,2024-05-17,2324,37.10,0.634855,-0.109729,-0.177782,0.489815,-0.274791,0.067852,0.072949,...,-0.422933,0.920795,0.753183,1.717425,-0.060892,1.491616,0.749619,0.0,0.0,0.0
0,2024-05-17,2356,53.50,0.445203,-0.154595,-0.119174,0.789926,-0.245183,0.013716,0.399226,...,-0.372287,0.611796,0.923609,1.490740,0.042677,1.446035,0.810067,0.0,0.0,0.0
0,2024-05-17,2376,322.50,0.558022,-0.017286,-0.171754,0.719022,-0.215026,-0.008799,0.371003,...,-0.485532,0.579057,0.605273,1.652529,0.235027,1.572498,0.743046,0.0,0.0,0.0
0,2024-05-17,2382,286.00,0.552969,-0.063527,-0.296229,0.685041,-0.188643,-0.059283,0.430857,...,-0.465231,0.663671,0.808204,1.373800,0.341852,1.517274,0.805467,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10,2024-06-05,3443,1580.00,0.658868,0.144944,-0.336764,0.221048,-0.081059,0.061236,0.755844,...,-0.853171,0.812033,0.671617,1.933807,0.929106,1.569586,0.768042,0.0,0.0,0.0
10,2024-06-05,3533,1595.00,0.437267,0.076646,-0.472099,0.134413,-0.246587,0.139379,0.952172,...,-0.951985,0.941479,0.660762,1.678192,1.046278,1.699484,0.738302,0.0,0.0,0.0
10,2024-06-05,8046,191.00,0.764475,-0.227007,-0.334844,0.193825,-0.492236,0.000845,0.332980,...,-0.642271,0.736638,0.879485,1.728126,0.342086,1.628090,0.917597,0.0,0.0,0.0
10,2024-06-05,9941,150.00,0.315398,0.051921,-0.219945,0.520131,-0.282850,0.118166,0.055047,...,-0.691445,0.533069,0.984331,1.672904,0.061797,1.708297,0.695229,0.0,0.0,0.0


In [43]:
e_trade_gym = StockTradingEnv(df = trade, turbulence_threshold = None,risk_indicator_col='vix', **env_kwargs)

trained_moedl =trained_ppo
df_account_value, df_actions ,df_balance= DRLAgent.DRL_prediction(
    model=trained_moedl, 
    environment = e_trade_gym)

print("==============Get Backtest Results===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+RESULTS_DIR+"/perf_stats_all_"+now+'.csv')


# df_account_value_ppo = pd.merge(df_account_value_ppo,df_account_value, on='date')
# df_balance_ppo = pd.merge(df_balance_ppo,df_balance, on='date')
# df_actions_ppo = pd.concat([df_actions_ppo, df_actions], ignore_index=False)    

df_account_value_ppo = df_account_value.copy()
df_balance_ppo = df_balance.copy()
df_actions_ppo = df_actions.copy()

hit end!
==============Get Backtest Results===========
Annual return         -0.443033
Cumulative returns    -0.025223
Annual volatility      0.183179
Sharpe ratio          -3.427163
Calmar ratio          -9.905540
Stability              0.630844
Max drawdown          -0.044726
Omega ratio            0.573184
Sortino ratio         -4.123885
Skew                        NaN
Kurtosis                    NaN
Tail ratio             0.653859
Daily value at risk   -0.025570
dtype: float64


c:\Users\yaowe\anaconda3\envs\myenv\lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


In [44]:
df_account_value

,date,account_value
0,2024-05-17,1.000000e+06
1,2024-05-21,1.002354e+06
2,2024-05-22,1.008116e+06
3,2024-05-23,1.009795e+06
4,2024-05-24,1.018649e+06
5,2024-05-28,1.004773e+06
6,2024-05-29,1.002558e+06
7,2024-05-30,9.798563e+05
8,2024-05-31,9.730894e+05
9,2024-06-04,9.876068e+05


In [45]:
df_actions

,1229,2324,2356,2376,2382,2395,2610,2618,3037,3443,3533,8046,9941,9945
date,,,,,,,,,,,,,,
2024-05-17,48,0,0,0,0,110,177,0,0,0,63,0,0,0
2024-05-21,48,0,0,0,0,110,177,0,0,0,63,0,0,0
2024-05-22,48,0,0,0,0,110,177,0,0,0,63,0,0,0
2024-05-23,48,0,0,0,0,110,177,0,0,0,63,0,0,0
2024-05-24,48,0,0,0,0,110,177,0,0,0,63,0,0,0
2024-05-28,48,0,0,0,0,110,177,0,0,0,63,0,0,0
2024-05-29,19,0,0,0,0,109,168,0,0,0,29,0,0,0
2024-05-30,-7,0,0,0,1,0,1,0,0,0,0,1,0,0
2024-05-31,-2,0,0,0,0,0,0,0,0,0,0,0,0,3
